In [43]:
#%pylab inline
import pandas as pd
import math

df = pd.DataFrame.from_csv('pain_log_201606100300.csv', index_col=2)
df['record date'] = pd.to_datetime(df['record date'])

def group_pain(fields):
    field = fields.pop()
    if fields:
        return df[field] | group_pain(fields)
    else:
        return df[field]

fields_calf = ['calf: ache', 'calf: sharp', 'calf: tight', 'calf: tingling']
fields_back = ['back: numb', 'back: sore', 'back: stiff']
fields_other = ['ankle: ache', 'leg: numb', 'thigh: ache']
fields_numb = ['calf: tingling', 'leg: numb', 'back: numb']
fields_pain = fields_calf + fields_back + fields_other

df['calf'] = group_pain(fields_calf)
df['back'] = group_pain(fields_back)
df['other'] = group_pain(fields_other)
df['numb'] = group_pain(fields_numb)
df['pain'] = group_pain(fields_pain)

# Overall

Show approx 95% confidence intervals for the various metrics.

In [40]:
def confidence_interval(ns):
    return ns.mean(), ns.mean() - (2 * ns.std()/math.sqrt(len(ns))), \
        ns.mean() + (2 * ns.std()/math.sqrt(len(ns)))


fields_for_ci = [col for col in list(df.columns) if col not in ['record date', 'record time', 'posture',
                                                                'activity', 'activity prev', 'notable']]

[ [field, confidence_interval(df[field])] for field in fields_for_ci]

[['ankle: ache',
  (0.12195121951219512, 0.076129929920598804, 0.16777250910379143)],
 ['calf: ache',
  (0.30243902439024389, 0.23812215243612006, 0.36675589634436773)],
 ['calf: sharp',
  (0.20000000000000001, 0.14398879663887962, 0.25601120336112038)],
 ['calf: tight',
  (0.22926829268292684, 0.1704058170943539, 0.2881307682714998)],
 ['calf: tingling',
  (0.23414634146341465, 0.17484950885238687, 0.29344317407444243)],
 ['leg: numb',
  (0.034146341463414637, 0.0087165577720464543, 0.059576125154782819)],
 ['thigh: ache',
  (0.16097560975609757, 0.10951413305069216, 0.21243708646150297)],
 ['buttock: ache',
  (0.24390243902439024, 0.18376957318412279, 0.30403530486465769)],
 ['back: numb',
  (0.029268292682926831, 0.0056655096439841741, 0.052871075721869484)],
 ['back: sore',
  (0.2097560975609756, 0.15274586849997665, 0.26676632662197458)],
 ['back: stiff',
  (0.12682926829268293, 0.080230521150342349, 0.17342801543502351)],
 ['should surgery',
  (0.41463414634146339, 0.345648118908

In [64]:
df[['posture', 'calf', 'pain', 'should surgery', 'is hopeful']].groupby(['posture']).mean()

,calf,pain,should surgery,is hopeful
posture,,,,
crouching,0.000000,0.000000,0.000000,1.000000
kneeling,1.000000,1.000000,0.500000,0.000000
lying,0.631579,0.789474,0.631579,0.578947
lying stomach,1.000000,1.000000,1.000000,0.500000
sitting,0.425000,0.625000,0.325000,0.775000
upright,0.695652,0.847826,0.391304,0.681159
walking,1.000000,1.000000,1.000000,0.666667


# Week-by-week comparison

Show approx 95% confidence intervals for the various metrics.

In [55]:
from datetime import datetime
df_week_one = df[df['record date'] < datetime(2016, 5, 29)]
print("Week one count: {0}".format(len(df_week_one)))
df_week_two = df[(df['record date'] >= datetime(2016, 5, 29)) & (df['record date'] < datetime(2016, 6, 4))]
print("Week two count: {0}".format(len(df_week_two)))
df_week_three = df[df['record date'] > datetime(2016, 6, 4)]
print("Week three count: {0}".format(len(df_week_three)))

def compare(field):
    return [field, { "week_1": confidence_interval(df_week_one[field]),
                     "week_2": confidence_interval(df_week_two[field]),
                     "week_3": confidence_interval(df_week_three[field]) }]

[ compare(field) for field in fields_for_ci]

Week one count: 80
Week two count: 59
Week three count: 55


[['ankle: ache',
  {'week_1': (0.0, 0.0, 0.0),
   'week_2': (0.3559322033898305, 0.23019452627239276, 0.48166988050726822),
   'week_3': (0.054545454545454543,
    -0.0072608425890034606,
    0.11635175167991255)}],
 ['calf: ache',
  {'week_1': (0.29999999999999999, 0.19688399061055037, 0.40311600938944958),
   'week_2': (0.3728813559322034, 0.24588940659998162, 0.49987330526442519),
   'week_3': (0.14545454545454545,
    0.049500265963329973,
    0.24140882494576094)}],
 ['calf: sharp',
  {'week_1': (0.28749999999999998, 0.18565779144658157, 0.38934220855341839),
   'week_2': (0.20338983050847459, 0.097682881219998505, 0.30909677979695066),
   'week_3': (0.072727272727272724,
    0.0020490679412690782,
    0.14340547751327637)}],
 ['calf: tight',
  {'week_1': (0.16250000000000001, 0.079489020094868634, 0.24551097990513138),
   'week_2': (0.25423728813559321, 0.13988734446386347, 0.36858723180732295),
   'week_3': (0.27272727272727271, 0.15151515151515152, 0.39393939393939392)}],
 ['ca